In [1]:
#6 - skirt
#7 -dress

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random
import os
import random
import yaml
import cv2
import shutil
import tqdm
import glob
import torch
from ultralytics import YOLO

C:\Users\USER\anaconda3\envs\preparation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
def img_preprocessing(img_path):
    # 한글 경로로 되어 있지 않을 때 1번, 한글 경로로 되어 있다면 2번을 사용함.
    # 1번
    #re_img = cv2.imread(img_path)
    # 2번
    img = imread(img_path)
    
    #전처리 부분
    # 여기서 416,416으로 이미지 크기를 조정함.
    # 이밖에 모자이크, 흑백 등의 전처리를 추가하고 싶다면 자유롭게 할 것.
    re_img = cv2.resize(img, (256, 256))
    return re_img
# 파일 위치가 한글 경로가 존재할 때 사용함(없다면 제외하고 해도 상관 없음)
def imwrite(filename, img, params=None):
    print("h")
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False
def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [4]:
path = os.getcwd()
img_path = os.path.join(path,"img_data")

In [5]:
for i in range(1,4):
    globals()[f"train_dir{i}"] = os.path.join(img_path,"train"+str(i))
    globals()[f"val_dir{i}"] = os.path.join(img_path,"val"+str(i))
    globals()[f"train_img_dir{i}"] = os.path.join(globals()[f"train_dir{i}"],"images")
    globals()[f"val_img_dir{i}"] = os.path.join(globals()[f"val_dir{i}"],"images")
    globals()[f"train_label_dir{i}"] = os.path.join(globals()[f"train_dir{i}"],"labels")
    globals()[f"val_label_dir{i}"] = os.path.join(globals()[f"val_dir{i}"],"labels")
    make_dir(globals()[f"train_dir{i}"])
    make_dir(globals()[f"val_dir{i}"])
    make_dir(globals()[f"train_img_dir{i}"])
    make_dir(globals()[f"val_img_dir{i}"])
    make_dir(globals()[f"train_label_dir{i}"])
    make_dir(globals()[f"val_label_dir{i}"])

In [6]:
for i in range(1,4):
    globals()[f"all_label_data{i}"] = [i.split(".")[0] for i in os.listdir(globals()[f"train_img_dir{i}"]) if i !="classes.txt"]

In [7]:
for k in range(1,4):
    train_label_dir = globals()[f"train_label_dir{k}"]
    train_img_dir = globals()[f"train_img_dir{k}"]
    for i in globals()[f"all_label_data{k}"]:
        label_path = os.path.join(train_label_dir, i+".txt")
        Blur_label_path = os.path.join(train_label_dir, i+"blur"+".txt")
        Color_label_path = os.path.join(train_label_dir, i+"color"+".txt")
        img_dir = os.path.join(train_img_dir, i+".jpg")
        New_blur_img_dir = os.path.join(train_img_dir,i+"blur"+".jpg")
        New_color_img_dir = os.path.join(train_img_dir,i+"color"+".jpg")
        f = open(label_path, 'r')
        data = f.read()
        f.close()
        h = open(Blur_label_path,'w')
        h2 = open(Color_label_path,'w')
        h.write(data)
        h2.write(data)
        h.close()
        h2.close()
        img = cv2.imread(img_dir)
        re_img = cv2.resize(img, (256, 256))
        blur = cv2.blur(re_img,(5,5))
        color = cv2.cvtColor(re_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(New_blur_img_dir,blur)
        cv2.imwrite(New_color_img_dir,color)
        if k==3:
            Gray_label_path = os.path.join(train_label_dir, i+"gray"+".txt")
            New_gray_img_dir = os.path.join(train_img_dir,i+"gray"+".jpg")
            h3 = open(Gray_label_path,'w')
            h3.write(data)
            h3.close()
            gray = cv2.cvtColor(re_img,cv2.COLOR_BGR2GRAY)
            cv2.imwrite(New_gray_img_dir,gray)